Parametri intrinseci della fotocamera

In [6]:
from roboflow import Roboflow
rf = Roboflow(api_key="FLw5QsWBWlIk65SnVgdn")
project = rf.workspace("labia-z0pkg").project("urban-semantic")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


KeyboardInterrupt: 

In [209]:
from roboflow import Roboflow
rf = Roboflow(api_key="FLw5QsWBWlIk65SnVgdn")
project = rf.workspace("usmanchaudhry622-gmail-com").project("traffic-and-road-signs")
version = project.version(1)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Traffic-and-Road-Signs-1 in yolov11:: 100%|██████████| 20012/20012 [00:07<00:00, 2540.40it/s]


In [8]:
from roboflow import Roboflow

rf = Roboflow(api_key="FLw5QsWBWlIk65SnVgdn")
project = rf.workspace("labia-z0pkg").project("urban-semantic")
yolo_model = project.version(1).model

loading Roboflow workspace...
loading Roboflow project...


In [210]:
model = YOLO("yolo11n.pt")

results = model.train(data="datasets/Traffic-and-Road-Signs-1/data.yaml", epochs=40, imgsz=640, device=0)

Ultralytics 8.3.63  Python-3.12.7 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=datasets/Traffic-and-Road-Signs-1/data.yaml, epochs=40, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=T

train: Scanning C:\Users\Edoardo\Desktop\università\Lab di IA\Progetto\urban-semantic-mapping\datasets\Traffic-and-Road-Signs-1\train\labels... 7092 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7092/7092 [00:09<00:00, 748.06it/s]


train: New cache created: C:\Users\Edoardo\Desktop\universit\Lab di IA\Progetto\urban-semantic-mapping\datasets\Traffic-and-Road-Signs-1\train\labels.cache


val: Scanning C:\Users\Edoardo\Desktop\università\Lab di IA\Progetto\urban-semantic-mapping\datasets\Traffic-and-Road-Signs-1\valid\labels... 1884 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1884/1884 [00:02<00:00, 634.24it/s]


val: New cache created: C:\Users\Edoardo\Desktop\universit\Lab di IA\Progetto\urban-semantic-mapping\datasets\Traffic-and-Road-Signs-1\valid\labels.cache
Plotting labels to runs\detect\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000303, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train2
Starting training for 40 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      40/40       2.6G     0.4867     0.2218      1.205          4        640: 100%|██████████| 444/444 [00:37<00:00, 11.91it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 59/59 [00:07<00:00,  7.91it/s]

                   all       1884       1886      0.915       0.95      0.931        0.8



40 epochs completed in 0.523 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 5.5MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics 8.3.63  Python-3.12.7 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
YOLO11n summary (fused): 238 layers, 2,587,807 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 59/59 [00:08<00:00,  6.94it/s]


                   all       1884       1886      0.923      0.956      0.937      0.803
-Road narrows on right         54         54      0.994          1      0.995      0.896
    50 mph speed limit         62         62      0.553          1      0.725      0.588
     Attention Please-        117        117      0.998          1      0.995      0.868
    Beware of children        106        106          1          1      0.995      0.887
CYCLE ROUTE AHEAD WARNING         54         54      0.994          1      0.995      0.871
Dangerous Left Curve Ahead         42         42      0.992          1      0.995      0.861
Dangerous Rright Curve Ahead         72         72      0.991          1      0.995      0.842
End of all speed and passing limits         48         48      0.993          1      0.995       0.84
              Give Way        107        107      0.997          1      0.995      0.856
Go Straight or Turn Right         79         79      0.996          1      0.995    

In [88]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import random
from ultralytics import YOLO
import open3d as o3d
from scipy.ndimage import gaussian_filter

Definizione della classe DataLoader 

In [89]:
class DataLoader(object):
    def __init__(self, path):
        self.main_path = path
        self.left_camera_images = sorted(os.listdir(os.path.join(self.main_path, "image_02/data")))
        self.right_camera_images = sorted(os.listdir(os.path.join(self.main_path, "image_03/data")))
        print(self.left_camera_images)
        self.frames = len(self.left_camera_images)

        calibration = pd.read_csv(os.path.join(self.main_path, "calib_cam_to_cam.txt"), delimiter=' ', header=None, index_col=0)
        #projection matrix
        self.P0 = np.array(calibration.loc['P_rect_02:'], dtype=np.float32).reshape((3,4))
        self.P1 = np.array(calibration.loc['P_rect_03:'], dtype=np.float32).reshape((3,4))

        #image loader
        self.left_images = []
        self.right_images = []

        for i, left in enumerate(self.left_camera_images):
            right = self.right_camera_images[i]
            self.left_images.append(cv2.imread((os.path.join(self.main_path, "image_02/data/")) + left))
            self.right_images.append(cv2.imread((os.path.join(self.main_path, "image_03/data/")) + right))
        
        self.first_image_left = self.left_images[0]
        self.first_image_right = self.right_images[0]
        self.second_image_left = self.left_images[1]
        self.second_image_right = self.right_images[1]

        self.image_height = self.left_images[0].shape[0]
        self.image_width = self.left_images[0].shape[1]

    def reset_frames(self):
        self.left_images = (cv2.imread((os.path.join(self.main_path, "image_02/data/")) + left, 0) for left in self.left_camera_images)
        self.right_images = (cv2.imread((os.path.join(self.main_path, "image_03/data/")) + right, 0) for right in self.right_camera_images)
        pass
        

In [90]:
with open("./initial_config.yaml", "r") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as error:
        print(error)

rgb_value = config['parameters']['rgb']
rectified_value = config['parameters']['rectified']
detector_name = config['parameters']['detector']
max_depth_value = config['parameters']['max_depth']

##stereo depth estimation

def disparity_mapping(left_image, right_image, rgb=rgb_value):
    if rgb:
        num_channels = 3
    else:
        num_channels = 1

    num_disparities = 6*16
    block_size = 7

    matcher = cv2.StereoSGBM_create(numDisparities = num_disparities, 
                                    minDisparity = 0, 
                                    blockSize = block_size, 
                                    P1=8 * num_channels * block_size ** 2,
                                    P2=32 * num_channels * block_size ** 2,
                                    mode = cv2.STEREO_SGBM_MODE_SGBM_3WAY)
    if rgb:
        left_image = cv2.cvtColor(left_image, cv2.COLOR_BGR2GRAY)
        right_image = cv2.cvtColor(right_image, cv2.COLOR_BGR2GRAY)
    
    left_image_disparity_map = matcher.compute(left_image, right_image).astype(np.float32)/16
    #plt.title("disparity mapping")
    #plt.imshow(left_image_disparity_map)

    return left_image_disparity_map

def decomposition(p):

    intrinsic_matrix, rotation_matrix, translation_vector, _, _, _, _ = cv2.decomposeProjectionMatrix(p)

    translation_vector = (translation_vector / translation_vector[3])[:3]

    return intrinsic_matrix, rotation_matrix, translation_vector

def depth_mapping(left_disparity_map, left_intrinsic, left_translation, right_translation, rectified=rectified_value):

    focal_length = left_intrinsic[0][0]

    if rectified:
        baseline = right_translation[0] - left_translation[0]
    else:
        baseline = left_translation[0] - right_translation[0]

    left_disparity_map[left_disparity_map == 0.0] = 0.1
    left_disparity_map[left_disparity_map == -1.0] = 0.1

    depth_map = np.ones(left_disparity_map.shape)
    depth_map = (focal_length * baseline) / left_disparity_map

    return depth_map

def stereo_depth(left_image, right_image, P0, P1, rgb=rgb_value):

    disp_map = disparity_mapping(left_image, right_image, rgb=rgb)

    l_intrinsic, l_rotation, l_translation = decomposition(P0)
    r_intrinsic, r_rotation, r_translation = decomposition(P1)

    depth = depth_mapping(disp_map, l_intrinsic, l_translation, r_translation)

    return depth

##stereo depth estimation


##feature extraction and matching

def feature_extractor(image, detector=detector_name,mask=None):

    if detector == 'sift':
        create_detector = cv2.SIFT_create()
    elif detector == 'orb':
        create_detector = cv2.ORB_create()

    keypoints, descriptors = create_detector.detectAndCompute(image, mask)

    return keypoints, descriptors

def feature_matching(first_descriptor, second_descriptor, detector=detector_name, k=2, distance_threshold=1.0):

    if detector == 'sift':
        feature_matcher = cv2.BFMatcher_create(cv2.NORM_L2, crossCheck = False)
    elif detector == 'orb':
        feature_matcher = cv2.BFMatcher_create(cv2.NORM_L2, crossCheck = False)
    matches = feature_matcher.knnMatch(first_descriptor, second_descriptor, k=k)
    filtered_matches = []
    for match1, match2 in matches:
        if match1.distance <= distance_threshold * match2.distance:
            filtered_matches.append(match1)

    return filtered_matches

def visualize_matches(first_image, second_image, keypoint_one, keypoint_two, matches):
    show_matches = cv2.drawMatches(first_image, keypoint_one, second_image, keypoint_two, matches, None, flags = 2)
    plt.figure(figsize=(15, 5), dpi=100)
    plt.imshow(show_matches)
    plt.show()

##feature extraction and matching

##motion estimation

def motion_estimation(matches, firstImage_keypoints, secondImage_keypoints, intrinsic_matrix, depth, max_depth=max_depth_value):
    """
    Estimating motion of the left camera from sequential imgaes 

    """
    rotation_matrix = np.eye(3)
    translation_vector = np.zeros((3, 1))

    # Only considering keypoints that are matched for two sequential frames
    image1_points = np.float32(
        [firstImage_keypoints[m.queryIdx].pt for m in matches])
    image2_points = np.float32(
        [secondImage_keypoints[m.trainIdx].pt for m in matches])

    cx = intrinsic_matrix[0, 2]
    cy = intrinsic_matrix[1, 2]
    fx = intrinsic_matrix[0, 0]
    fy = intrinsic_matrix[1, 1]

    points_3D = np.zeros((0, 3))
    outliers = []

    # Extract depth information to build 3D positions
    for indices, (u, v) in enumerate(image1_points):
        z = depth[int(v), int(u)]

        # We will not consider depth greater than max_depth
        if z > max_depth:
            outliers.append(indices)
            continue

        # Using z we can find the x,y points in 3D coordinate using the formula
        x = z*(u-cx)/fx
        y = z*(v-cy)/fy

        # Stacking all the 3D (x,y,z) points
        points_3D = np.vstack([points_3D, np.array([x, y, z])])

    # Deleting the false depth points
    image1_points = np.delete(image1_points, outliers, 0)
    image2_points = np.delete(image2_points, outliers, 0)

    # Apply Ransac Algorithm to remove outliers
    _, rvec, translation_vector, _ = cv2.solvePnPRansac(
        points_3D, image2_points, intrinsic_matrix, None)

    rotation_matrix = cv2.Rodrigues(rvec)[0]

    return rotation_matrix, translation_vector, image1_points, image2_points



In [91]:
yolo_model = YOLO('yolo11n.pt') 
yolo_model.names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [92]:
classtoexclude = [0, 1, 2, 3, 5, 6, 7, 8]

In [93]:
results = yolo_model(["videoreferences/2011_09_26_drive_0005_sync/image_02/data/0000000050.png"])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()


0: 224x640 1 person, 3 bicycles, 4 cars, 31.5ms
Speed: 2.0ms preprocess, 31.5ms inference, 2.0ms postprocess per image at shape (1, 3, 224, 640)


definizione della funzione yolo_elaboration da inserire nel ciclo della gestione dei frame del video

In [94]:
def yolo_elaboration(frame):
    detection_info = []
    detection_id = []
    #carica l'immagine
    #img = cv2.imread(frame)

    #esegui il rilevamento con yolov8
    yolo_results = yolo_model.track(frame, persist=True)

    if not yolo_results[0]:
        return detection_info, detection_id 
    

    #ottieni i risultati del rilevamento (results[0] indica le informazioni ottenute dall'unica immagine in input, se ci fossero piu di una immagine allora sarebbe results[i] con i = 1...n)
    detection = yolo_results[0].boxes.xyxy #coordinate della bounding box
    confidences = yolo_results[0].boxes.conf #confidenza della rilevazione
    class_ids = yolo_results[0].boxes.cls #ID delle classi di appartenenza dell'oggetto
    obj_ids = yolo_results[0].boxes.id #ID dell'oggetto
    
    #visualizza i risultati
    
    for i, (box, conf, cls_id, obj_id) in enumerate(zip(detection, confidences, class_ids, obj_ids)):
        if cls_id in classtoexclude:
            continue
        x1, y1, x2, y2 = map(int, box) #estrai le coordinate della bounding box

        label = f"{yolo_model.names[int(cls_id)]} ({conf:.2f})" #viene creata un'etichetta contenente il nome della classe (poiche class_ids e' un intero che grazie a model.names[cls_id] viene convertito nella corrispondente classe) e la confidenza

        detection_info.append({
            #'frame': capture.get(cv2.CAP_PROP_POS_FRAMES),
            'id': obj_id,
            'label': label,
            'confidence': conf,
            'class': cls_id,
            'x1': x1,
            'x2': x2,
            'y1': y1,
            'y2': y2
        })
        detection_id.append(obj_id)
    annotated_frame = yolo_results[0].plot()
    #plt.title("yolo")
    #plt.imshow(annotated_frame)
    #cv2.waitKey(0)
    #cv2.imshow("si", annotated_frame)
    #cv2.destroyAllWindows()
    return detection_info, detection_id

combinazione dei risultati delle due funzioni precedenti per calcolare la distanza stimata degli oggetti rilevati nel frame

In [95]:
def elaboration(depth_map, current_detection, old_detection, old_detection_id, intrinsic_matrix, transformation_matrix):
    if not current_detection:
        return old_detection
    detection_info = current_detection
    cx = intrinsic_matrix[0, 2]
    cy = intrinsic_matrix[1, 2]
    fx = intrinsic_matrix[0, 0]
    fy = intrinsic_matrix[1, 1]
    for object in detection_info:
        obj_id = object['id'] 
        obj_class = object['class']
        #se l'oggetto rilevato e' stato gia scansionato precedentemente, ne miglioro la posizione aggiungendo le coordinate dei pixel della scansione nuova
        if obj_id in old_detection_id:
            
            object_data = []
            for obj in old_detection:
                if obj['id'] == obj_id:
                    object_data = obj['3dcoord']
                    break
            
            label = object['label']
            confidence = object['confidence']
            x1 = object['x1']
            x2 = object['x2']
            y1 = object['y1']
            y2 = object['y2']
            #if confidence < 0.20:
                #continue
            for y in range(y1, y2):
                for x in range(x1, x2):
                    #campiono il 20% dei pixel all'interno della bounding box
                    if random.random() > 0.20:
                        continue
                    depth = depth_map[y,x]
                    if depth > 100:
                        continue
                    real_x = ((x - cx) * depth)/fx
                    real_y = ((y - cy) * depth)/fy
                    obj_coords = np.array([real_x, real_y, depth, 1])
                    world_obj_coords = transformation_matrix.dot(obj_coords)
                    object_data.append(world_obj_coords)
            for i, obj in enumerate(old_detection):
                if obj['id'] == obj_id:
                    old_detection[i] = {'id': obj_id,
                                        'class': obj_class,
                                        'label': label,
                                        'confidence': confidence,
                                        '3dcoord': object_data}
            continue
        #se l'oggetto scansionato non era stato scansionato precedentemente, aggiungo un nuovo elemento nella lista old_detection che contiene varie informazioni tra cui l'id e le coordinate dei pixel del nuovo oggetto
        label = object['label']
        confidence = object['confidence']
        x1 = object['x1']
        x2 = object['x2']
        y1 = object['y1']
        y2 = object['y2']
        #if confidence < 0.20:
            #continue
        object_data = []
        for y in range(y1, y2):
            for x in range(x1, x2):
                #campiono il 20% dei pixel all'interno della bounding box
                if random.random() > 0.2:
                    continue
                depth = depth_map[y,x]
                if depth > 100:
                    continue
                real_x = ((x - cx) * depth)/fx
                real_y = ((y - cy) * depth)/fy
                obj_coords = np.array([real_x, real_y, depth, 1])
                world_obj_coords = transformation_matrix.dot(obj_coords)
                #print(world_obj_coords)
                object_data.append(world_obj_coords)
        #if len(object_data) > 1:
            #object_data = pointCleaning(object_data, 0.5)
        old_detection.append({
            'id' : obj_id,
            'class': obj_class,
            'label' : label,
            'confidence' : confidence,
            '3dcoord' : object_data
        })
    return old_detection
        


In [96]:
from scipy.spatial import cKDTree

def pointCleaning(points, threshold):
    var = 0
    
    kdtree = cKDTree(points)
    cleaned_points = []

    for i, point in enumerate(points):
        neighbour = kdtree.query(point, k=100, distance_upper_bound=threshold)

        distance = neighbour[0]
        idxs = neighbour[1]

        distance = distance[distance < threshold]

        if len(distance) > 50:
            cleaned_points.append(point)
    if var == 1:
        for point in cleaned_points:
            point = np.array([point[0], point[1], point[2], 1])
    return cleaned_points

In [97]:
def get_next_filename(folder, base_name, ext):
    index = 1
    while True:
        filename = base_name + str(index) + ext
        filepath = os.path.join(folder, filename)
        if not os.path.exists(filepath):
            return filepath
        index += 1

In [98]:
##TODO: sistemare i colori a seconda della classe includendo una pulizia dei punti

def threedMapConversion(detection_info, camera_coordinates,):
    points = []
    for object in detection_info:
        threedcoords = [coord[:3] for coord in object['3dcoord']]
        points.extend(threedcoords)
    if not points:
        print("Non sono stati scansionati oggetti")
        return
    print("i punti sono", points)
    points = pointCleaning(points, 0.2)
    points.extend(camera_coordinates)
    points = np.array(points)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    o3d.visualization.draw_geometries([pcd])

    output_path = get_next_filename("renders", "output", ".ply")
    print(output_path)
    o3d.io.write_point_cloud(output_path, pcd)


In [99]:
with open("./initial_config.yaml", "r") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as error:
        print(error)

detector_name = config['parameters']['detector']
subset = config['parameters']['subset']
threshold = config['parameters']['distance_threshold']

def odometry(data_handler, detector = detector_name, mask = None, subset = subset, plot = True):

    if subset is not None:
        num_frames = subset
    else:
        num_frames = data_handler.frames

    # Create a homogeneous matrix
    homo_matrix = np.eye(4)
    trajectory = np.zeros((num_frames, 3, 4))
    trajectory[0] = homo_matrix[:3, :]
    camera_coordinates = []

    # From projection matrix retrieve the left camera's intrinsic matrix
    left_intrinsic_matrix, _, _ = decomposition(data_handler.P0)

    #yolo
    detection = []
    detection_id = []

    # Loop to iterate all the frames
    for i in range(num_frames - 1):

        image_left = data_handler.left_images[i]
        image_right = data_handler.right_images[i]
        next_image = data_handler.left_images[i + 1]

        # Estimating the depth of an image
        depth = stereo_depth(image_left, image_right, P0=data_handler.P0, P1=data_handler.P1)
        #plt.title("Stereo Depth Mapping")
        #plt.figure(figsize=(12,6))
        #plt.imshow(depth)
        #print(depth)
        
        # Keypoints and Descriptors of two sequential images of the left camera
        keypoint_left_first, descriptor_left_first = feature_extractor(image_left, detector, mask)
        keypoint_left_next, descriptor_left_next = feature_extractor(next_image, detector, mask)

        # Use feature (e.g. SIFT or ORB) detector to match features
        matches = feature_matching(descriptor_left_first, descriptor_left_next, detector=detector, distance_threshold=threshold)

        # Estimate motion between sequential images of the left camera
        rotation_matrix, translation_vector, _, _ = motion_estimation(matches, keypoint_left_first, keypoint_left_next, left_intrinsic_matrix, depth)

        # Initialise a homogeneous matrix (4X4)
        Transformation_matrix = np.eye(4)

        # Build the Transformation matrix using rotation matrix and translation vector from motion estimation function
        Transformation_matrix[:3, :3] = rotation_matrix
        Transformation_matrix[:3, 3] = translation_vector.T

        # Transformation wrt. world coordinate system
        homo_matrix = homo_matrix.dot(np.linalg.inv(Transformation_matrix))

        # Append the pose of camera in the trajectory array
        trajectory[i+1, :, :] = homo_matrix[:3, :]
        camera_coordinates.append((trajectory[i,0,3], trajectory[i,1,3], trajectory[i,2,3]))

        #yolo
        new_detection, new_detection_id = yolo_elaboration(image_left)

        #elaboration
        detection = elaboration(depth, new_detection, detection, detection_id, left_intrinsic_matrix, homo_matrix)

        if new_detection_id is not None:
            detection_id.extend(new_detection_id)
        if i % 10 == 0:
            print(f'{i} frames have been computed')

        if i == num_frames - 2:
            print('All frames have been computed')
            break
    threedMapConversion(detection, camera_coordinates)
    return trajectory


In [100]:
path = "videoreferences/2011_09_26_drive_0005_sync"

data_handler = DataLoader(path)

trajectory = odometry(data_handler)

['0000000000.png', '0000000001.png', '0000000002.png', '0000000003.png', '0000000004.png', '0000000005.png', '0000000006.png', '0000000007.png', '0000000008.png', '0000000009.png', '0000000010.png', '0000000011.png', '0000000012.png', '0000000013.png', '0000000014.png', '0000000015.png', '0000000016.png', '0000000017.png', '0000000018.png', '0000000019.png', '0000000020.png', '0000000021.png', '0000000022.png', '0000000023.png', '0000000024.png', '0000000025.png', '0000000026.png', '0000000027.png', '0000000028.png', '0000000029.png', '0000000030.png', '0000000031.png', '0000000032.png', '0000000033.png', '0000000034.png', '0000000035.png', '0000000036.png', '0000000037.png', '0000000038.png', '0000000039.png', '0000000040.png', '0000000041.png', '0000000042.png', '0000000043.png', '0000000044.png', '0000000045.png', '0000000046.png', '0000000047.png', '0000000048.png', '0000000049.png', '0000000050.png', '0000000051.png', '0000000052.png', '0000000053.png', '0000000054.png', '00000000